In [1]:
%matplotlib inline

import os
import sys
import utils
import random
import dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

#import tensorflow.contrib.eager as tfe
#tfe.enable_eager_execution()
sess=tf.Session()

In [2]:
# load data
X=dataset.load_dataset(num_of_examples=10)
Y=dataset.load_labels(num_of_examples=10)

[fetch_renders_from_disk] fetching renders from ShapeNetRendering ... 
[construct_path_lists] parsing dir ShapeNetRendering for ['.png'] ...
[construct_path_lists] parsing dir ShapeNetVox32 for ['.binvox'] ...


In [3]:
# encoder network
cur_tensor=tf.convert_to_tensor(X,dtype=tf.float32)
print(cur_tensor.shape)
k_s = [3,3]
conv_filter_count = [96, 128, 256, 256, 256, 256]
for i in range(6): 
    ks=[7,7]if i is 0 else k_s  
    with tf.name_scope("encoding_block"):
        cur_tensor=tf.layers.conv2d(cur_tensor,filters=conv_filter_count[i],padding='SAME',kernel_size= k_s,activation=None)
        cur_tensor=tf.layers.max_pooling2d(cur_tensor,2,2)
        cur_tensor=tf.nn.relu(cur_tensor)
        print(cur_tensor.shape)

# flatten tensor
cur_tensor=tf.contrib.layers.flatten(cur_tensor)
cur_tensor=tf.contrib.layers.fully_connected(cur_tensor,1024,activation_fn=None)
print(cur_tensor.shape)

(10, 137, 137, 3)
(10, 68, 68, 96)
(10, 34, 34, 128)
(10, 17, 17, 256)
(10, 8, 8, 256)
(10, 4, 4, 256)
(10, 2, 2, 256)
(10, 1024)


In [4]:
# 3D_Convolutional_LSTM 

with tf.name_scope("3D_LSTM"):
    #x=tf.placeholder(tf.float32,[None,1024])
    x=cur_tensor
    h_t=tf.Variable(tf.zeros([4,4,4,1,256]),name="hidden_state")
    s_t=tf.Variable(tf.zeros([4,4,4,1,256]),name="cell_state")

    # W,U,b for f,i,s,o
    W=[tf.Variable(tf.zeros([1024,256]),name="W%d"%(i)) for i in range(4)]
    U=[tf.Variable(tf.zeros([3,3,3,256,256]),name="U%d"%(i)) for i in range(4)]
    b=[tf.Variable(tf.zeros([1,256]),name="b%d"%(i)) for i in range(4)]

    Wx=[tf.matmul(x, W[i]) for i in range(4)]
    Uh=[tf.nn.convolution(h_t,U[i],padding="SAME") for i in range(4)]


    f_t=tf.sigmoid(Wx[0] + Uh[0] + b[0])
    i_t=tf.sigmoid(Wx[1] + Uh[1] + b[1])
    o_t=tf.sigmoid(Wx[2]+ Uh[2] + b[2])
    s_t=tf.multiply(f_t,s_t) + tf.multiply(i_t,tf.tanh(Wx[3]+ Uh[3] + b[3]))
    h_t=tf.tanh(s_t)

    cur_tensor=tf.transpose(h_t,[3,0,1,2,4])
    print(cur_tensor.shape)

(10, 4, 4, 4, 256)


In [5]:
# decoding network
# batch_size=cur_tensor.shape.as_list()[0]
# cur_tensor=tf.reshape(cur_tensor,[batch_size,2,2,2,-1])
# print(cur_tensor.shape)
k_s = [3,3,3]
deconv_filter_count = [128, 128, 128, 64, 32, 2]
cur_tensor=utils.unpool(cur_tensor)
print(cur_tensor.shape)
for i in range(2,4): 
    with tf.name_scope("decoding_block"):
            cur_tensor=tf.layers.conv3d_transpose(cur_tensor,padding='SAME',filters=deconv_filter_count[i],kernel_size= k_s,activation=None)
            cur_tensor=utils.unpool(cur_tensor)
            cur_tensor=tf.nn.relu(cur_tensor)
            print(cur_tensor.shape)
            
for i in range(4,6): 
    with tf.name_scope("decoding_block_without_unpooling"):
            cur_tensor=tf.layers.conv3d_transpose(cur_tensor,padding='SAME',filters=deconv_filter_count[i],kernel_size= k_s,activation=None)
            cur_tensor=tf.nn.relu(cur_tensor)
            print(cur_tensor.shape)

(10, 8, 8, 8, 256)
(10, 16, 16, 16, 128)
(10, 32, 32, 32, 64)
(10, 32, 32, 32, 32)
(10, 32, 32, 32, 2)


In [8]:
#loss softmax
p=cur_tensor

tf.reduce_sum(tf.multiply(tf.log(p),Y)+tf.multiply(tf.log(1-p),Y))

TypeError: Expected float32 passed to parameter 'y' of op 'Mul', got [<binvox_rw.Voxels object at 0x1265ed1d0>, <binvox_rw.Voxels object at 0x12f05a8d0>, <binvox_rw.Voxels object at 0x12f05aa58>, <binvox_rw.Voxels object at 0x12f05a898>, <binvox_rw.Voxels object at 0x12f05a9b0>, <binvox_rw.Voxels object at 0x12f05a908>, <binvox_rw.Voxels object at 0x12f05a940>, <binvox_rw.Voxels object at 0x12f05a5c0>, <binvox_rw.Voxels object at 0x12f05a978>, <binvox_rw.Voxels object at 0x12f05a9e8>] of type 'list' instead.

In [ ]:
# serialize graph for tensorboard
os.system("rm -rf ./logs/*")
writer = tf.summary.FileWriter("./logs/")
writer.add_graph(sess.graph)
os.system("sh tensorboard.sh")